# ETL Project Breweries and Census Data
### The purpose of this project is to practice extracting, transforming, and loading data from two sources. 

### The goal is to provide data for US breweries per state population.


In [1]:
# import Dependencies
import pandas as pd
import requests
import pymongo

# (E)xtraction
### Sources used for this project:

#### From Kaggle:
* Breweries: https://www.kaggle.com/brkurzawa/us-breweries#breweries_us.csv

#### From US Census Bureau:
* Population: https://www.census.gov/newsroom/press-kits/2018/pop-estimates-national-state.html![image.png](attachment:image.png)

In [2]:
#read breweries csv into dataframe and display head.

breweries_df = pd.read_csv("Resources/breweries_us.csv")
breweries_df.head()


,brewery_name,type,address,website,state,state_breweries
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284


In [3]:
#read census csv into dataframe and display head.
census_df = pd.read_csv("Resources/nst-est2018-popchg2010_2018.csv")
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,...,NRANK_NPCHG2018,NRANK_PPCHG2010,NRANK_PPCHG2011,NRANK_PPCHG2012,NRANK_PPCHG2013,NRANK_PPCHG2014,NRANK_PPCHG2015,NRANK_PPCHG2016,NRANK_PPCHG2017,NRANK_PPCHG2018
0,10,0,0,0,United States,308758105,309326085,311580009,313874218,316057727,...,X,X,X,X,X,X,X,X,X,X
1,20,1,0,0,Northeast Region,55318430,55380645,55600532,55776729,55907823,...,4,3,3,3,4,4,4,4,4,4
2,20,2,0,0,Midwest Region,66929743,66974749,67152631,67336937,67564135,...,3,4,4,4,3,3,3,3,3,3
3,20,3,0,0,South Region,114563045,114867066,116039399,117271075,118393244,...,1,1,1,1,1,1,1,1,1,1
4,20,4,0,0,West Region,71946887,72103625,72787447,73489477,74192525,...,2,2,2,2,2,2,2,2,2,2


# (T)ransformation

### Transform breweries data

In [4]:
# explore column names
breweries_df.columns

Index(['brewery_name', 'type', 'address', 'website', 'state',
       'state_breweries'],
      dtype='object')

In [5]:
# select specific columns
 
breweries_col = ['state',
       'state_breweries']
breweries = breweries_df[breweries_col].copy()

# Rename columns
breweries = breweries.rename(columns={'state_breweries': 'count'})

# Change state names to lower-case to match the two datasets

breweries['state']=breweries['state'].str.lower() 

# Replace '-' with ' ' in state namnes
breweries['state']=breweries['state'].str.replace(r'-',' ')

# remove duplicates
breweries.drop_duplicates(inplace=True)
breweries


,state,count
0,california,284
284,nevada,18
302,wyoming,8
310,alabama,6
316,connecticut,11
327,iowa,13
340,missouri,21
361,nebraska,15
376,alaska,14
390,arizona,34


In [6]:
# explore columns in census dataframe.
census_df.columns

Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'NAME', 'ESTIMATESBASE2010',
       'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012',
       'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015',
       'POPESTIMATE2016', 'POPESTIMATE2017', 'POPESTIMATE2018', 'NPOPCHG_2010',
       'NPOPCHG_2011', 'NPOPCHG_2012', 'NPOPCHG_2013', 'NPOPCHG_2014',
       'NPOPCHG_2015', 'NPOPCHG_2016', 'NPOPCHG_2017', 'NPOPCHG_2018',
       'PPOPCHG_2010', 'PPOPCHG_2011', 'PPOPCHG_2012', 'PPOPCHG_2013',
       'PPOPCHG_2014', 'PPOPCHG_2015', 'PPOPCHG_2016', 'PPOPCHG_2017',
       'PPOPCHG_2018', 'NRANK_ESTBASE2010', 'NRANK_POPEST2010',
       'NRANK_POPEST2011', 'NRANK_POPEST2012', 'NRANK_POPEST2013',
       'NRANK_POPEST2014', 'NRANK_POPEST2015', 'NRANK_POPEST2016',
       'NRANK_POPEST2017', 'NRANK_POPEST2018', 'NRANK_NPCHG2010',
       'NRANK_NPCHG2011', 'NRANK_NPCHG2012', 'NRANK_NPCHG2013',
       'NRANK_NPCHG2014', 'NRANK_NPCHG2015', 'NRANK_NPCHG2016',
       'NRANK_NPCHG2017', 'NRANK_NPCHG2

In [7]:
#Tranform and clean-up

census_df.head(10)



,SUMLEV,REGION,DIVISION,STATE,NAME,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,...,NRANK_NPCHG2018,NRANK_PPCHG2010,NRANK_PPCHG2011,NRANK_PPCHG2012,NRANK_PPCHG2013,NRANK_PPCHG2014,NRANK_PPCHG2015,NRANK_PPCHG2016,NRANK_PPCHG2017,NRANK_PPCHG2018
0,10,0,0,0,United States,308758105,309326085,311580009,313874218,316057727,...,X,X,X,X,X,X,X,X,X,X
1,20,1,0,0,Northeast Region,55318430,55380645,55600532,55776729,55907823,...,4,3,3,3,4,4,4,4,4,4
2,20,2,0,0,Midwest Region,66929743,66974749,67152631,67336937,67564135,...,3,4,4,4,3,3,3,3,3,3
3,20,3,0,0,South Region,114563045,114867066,116039399,117271075,118393244,...,1,1,1,1,1,1,1,1,1,1
4,20,4,0,0,West Region,71946887,72103625,72787447,73489477,74192525,...,2,2,2,2,2,2,2,2,2,2
5,40,3,6,1,Alabama,4780138,4785448,4798834,4815564,4830460,...,27,37,38,34,33,35,36,34,34,34
6,40,4,9,2,Alaska,710249,713906,722038,730399,737045,...,45,2,8,11,18,48,37,22,46,49
7,40,4,8,4,Arizona,6392288,6407774,6473497,6556629,6634999,...,4,15,10,7,7,7,9,9,6,4
8,40,3,7,5,Arkansas,2916028,2921978,2940407,2952109,2959549,...,30,22,28,31,38,31,30,26,24,28
9,40,4,9,6,California,37254523,37320903,37641823,37960782,38280824,...,3,26,17,19,20,16,16,20,23,25


In [8]:
# drop rows that are not states.
census_df = census_df[census_df['STATE'] > 0]
census_df

,SUMLEV,REGION,DIVISION,STATE,NAME,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,...,NRANK_NPCHG2018,NRANK_PPCHG2010,NRANK_PPCHG2011,NRANK_PPCHG2012,NRANK_PPCHG2013,NRANK_PPCHG2014,NRANK_PPCHG2015,NRANK_PPCHG2016,NRANK_PPCHG2017,NRANK_PPCHG2018
5,40,3,6,1,Alabama,4780138,4785448,4798834,4815564,4830460,...,27,37,38,34,33,35,36,34,34,34
6,40,4,9,2,Alaska,710249,713906,722038,730399,737045,...,45,2,8,11,18,48,37,22,46,49
7,40,4,8,4,Arizona,6392288,6407774,6473497,6556629,6634999,...,4,15,10,7,7,7,9,9,6,4
8,40,3,7,5,Arkansas,2916028,2921978,2940407,2952109,2959549,...,30,22,28,31,38,31,30,26,24,28
9,40,4,9,6,California,37254523,37320903,37641823,37960782,38280824,...,3,26,17,19,20,16,16,20,23,25
10,40,4,8,8,Colorado,5029316,5048281,5121771,5193721,5270482,...,8,6,4,5,5,4,4,10,8,7
11,40,1,1,9,Connecticut,3574147,3579125,3588023,3594395,3594915,...,44,30,42,43,50,46,50,48,44,43
12,40,3,5,10,Delaware,897934,899595,907316,915188,923638,...,32,24,18,18,17,13,15,16,17,12
13,40,3,5,11,District of Columbia,601766,605085,619602,634725,650431,...,35,1,1,1,2,2,2,7,10,15
14,40,3,5,12,Florida,18804580,18845785,19093352,19326230,19563166,...,2,19,6,8,6,5,5,2,5,5


In [9]:
# We are only interested in state name and most recent population, i,e, 2018 population.
# Rank data, change in population was not relevant to our projects.

census_columns= ['NAME', 'POPESTIMATE2018']
census = census_df[census_columns].copy()
census = census.rename(columns={'NAME': 'state','POPESTIMATE2018':'pop_2018'})

# Change state names to lower-case to match the two datasets
census['state'] = census['state'].str.lower()

# reset index to start at zero
census = census.reset_index()

# drop the index column
del census['index']

census


,state,pop_2018
0,alabama,4887871
1,alaska,737438
2,arizona,7171646
3,arkansas,3013825
4,california,39557045
5,colorado,5695564
6,connecticut,3572665
7,delaware,967171
8,district of columbia,702455
9,florida,21299325


In [10]:
# change district of columnbia in the census dataframe to match the breweries dataframe
census.iloc[8,0] 
census.iloc[8,0] = 'washington dc'
census

,state,pop_2018
0,alabama,4887871
1,alaska,737438
2,arizona,7171646
3,arkansas,3013825
4,california,39557045
5,colorado,5695564
6,connecticut,3572665
7,delaware,967171
8,washington dc,702455
9,florida,21299325


In [11]:
# Merge breweries and census data into a single dataframe, merging by state name

breweries_by_state = breweries.merge(census, left_on='state', right_on='state', how='outer')
breweries_by_state.sort_values('state')

,state,count,pop_2018
3,alabama,6.0,4887871
8,alaska,14.0,737438
9,arizona,34.0,7171646
10,arkansas,7.0,3013825
0,california,284.0,39557045
11,colorado,182.0,5695564
4,connecticut,11.0,3572665
12,delaware,14.0,967171
13,florida,99.0,21299325
14,georgia,47.0,10519475


In [12]:
# replace NaN with zero for Mississippi because it had no data.
breweries_by_state.iloc[51,1]

breweries_by_state.iloc[51,1]=0

breweries_by_state

,state,count,pop_2018
0,california,284.0,39557045
1,nevada,18.0,3034392
2,wyoming,8.0,577737
3,alabama,6.0,4887871
4,connecticut,11.0,3572665
5,iowa,13.0,3156145
6,missouri,21.0,6126452
7,nebraska,15.0,1929268
8,alaska,14.0,737438
9,arizona,34.0,7171646


# (L)oad

### for this project we elected to use MongoDB

In [13]:
# Use Pymongo to set up mongo connection
conn= 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [14]:
# Define a database and collection
db=client.breweries_db
breweries =db.breweries

In [15]:
# Convert dataframe to a dictionary before inserting into the collection
breweries_dict = breweries_by_state.to_dict('records')
breweries_dict

[{'state': 'california', 'count': 284.0, 'pop_2018': 39557045},
 {'state': 'nevada', 'count': 18.0, 'pop_2018': 3034392},
 {'state': 'wyoming', 'count': 8.0, 'pop_2018': 577737},
 {'state': 'alabama', 'count': 6.0, 'pop_2018': 4887871},
 {'state': 'connecticut', 'count': 11.0, 'pop_2018': 3572665},
 {'state': 'iowa', 'count': 13.0, 'pop_2018': 3156145},
 {'state': 'missouri', 'count': 21.0, 'pop_2018': 6126452},
 {'state': 'nebraska', 'count': 15.0, 'pop_2018': 1929268},
 {'state': 'alaska', 'count': 14.0, 'pop_2018': 737438},
 {'state': 'arizona', 'count': 34.0, 'pop_2018': 7171646},
 {'state': 'arkansas', 'count': 7.0, 'pop_2018': 3013825},
 {'state': 'colorado', 'count': 182.0, 'pop_2018': 5695564},
 {'state': 'delaware', 'count': 14.0, 'pop_2018': 967171},
 {'state': 'florida', 'count': 99.0, 'pop_2018': 21299325},
 {'state': 'georgia', 'count': 47.0, 'pop_2018': 10519475},
 {'state': 'hawaii', 'count': 10.0, 'pop_2018': 1420491},
 {'state': 'idaho', 'count': 28.0, 'pop_2018': 1754

In [16]:
# insert dictionary into breweries_db
breweries.insert_many(breweries_dict)

# Heatmap

In [17]:
# Verify that mongoDB collection was inserted properly
breweries = db.breweries.find({'state': 'california'})
breweries =list(breweries)
breweries

[{'_id': ObjectId('5dd5e34c25528bbd6b410624'),
  'state': 'california',
  'count': 284.0,
  'pop_2018': 39557045},
 {'_id': ObjectId('5dd5e787a73e509a1f3a82bc'),
  'state': 'california',
  'count': 284.0,
  'pop_2018': 39557045}]